<a href="https://colab.research.google.com/github/Thiago3610/BD-Iris/blob/main/notebooks/Teste_de_metricas_com_category_name_dividida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este projeto, os dados utilizados foram obtidos do Kaggle - https://www.kaggle.com/competitions/mercari-price-suggestion-challenge/overview 

Thiago Junior Charles 25/09/2022

Fiz a utilização das funções .astype e .cat.codes, para transformar as colunas que continham texto em numeros, assim diminuindo o uso de ram e velocidade dos modelos, visto que não iria utilizar a função .vectorizer. 

### Importação de Bibliotecas

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import random
import seaborn as sns



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
from sklearn.model_selection import train_test_split


### Leitura do arquivo

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/modulo 6/train.tsv', sep = '\t')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/modulo 6/test.tsv', sep = '\t')

In [4]:
display(train.head())
display(train.tail())

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
1482530,1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl..."
1482531,1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t
1482532,1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape."
1482533,1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...
1482534,1482534,Brand new lux de ville wallet,1,Women/Women's Accessories/Wallets,NaN,22.0,0,"New with tag, red with sparkle. Firm price, no..."


In [5]:
display(test.head())
display(test.tail())

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
693354,693354,Quartz crystal on Flint stone,1,Home/Home Décor/Home Décor Accents,NaN,0,Flint/Quartz cluster. Self mined ✨measures 3x2...
693355,693355,It Cosmetics - Travel Bundle,1,Beauty/Makeup/Makeup Sets,IT Cosmetics,1,It Cosmetics travel bundle. Includes: Brow pow...
693356,693356,Galaxy S8 hard shell case,1,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,1,New. Free shipping Basstop case
693357,693357,Hi low floral kimono,2,Women/Swimwear/Cover-Ups,NaN,0,Floral kimono. Tropical print. Open front. Hi ...
693358,693358,"FREESHIP 2 Floral Scrub Tops, medium.",2,Women/Tops & Blouses/T-Shirts,NaN,1,2 Floral scrub tops. Worn less than 5 times ea...


### Indexes redundantes

In [6]:
print(train.describe())
print('------------------------------------------\n')
print(train.info())

           train_id  item_condition_id         price      shipping
count  1.482535e+06       1.482535e+06  1.482535e+06  1.482535e+06
mean   7.412670e+05       1.907380e+00  2.673752e+01  4.472744e-01
std    4.279711e+05       9.031586e-01  3.858607e+01  4.972124e-01
min    0.000000e+00       1.000000e+00  0.000000e+00  0.000000e+00
25%    3.706335e+05       1.000000e+00  1.000000e+01  0.000000e+00
50%    7.412670e+05       2.000000e+00  1.700000e+01  0.000000e+00
75%    1.111900e+06       3.000000e+00  2.900000e+01  1.000000e+00
max    1.482534e+06       5.000000e+00  2.009000e+03  1.000000e+00
------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null 

In [7]:
print(test.describe())
print('------------------------------------------\n')
print(test.info())

             test_id  item_condition_id       shipping
count  693359.000000      693359.000000  693359.000000
mean   346679.000000           1.906102       0.447719
std    200155.646985           0.903378       0.497260
min         0.000000           1.000000       0.000000
25%    173339.500000           1.000000       0.000000
50%    346679.000000           2.000000       0.000000
75%    520018.500000           3.000000       1.000000
max    693358.000000           5.000000       1.000000
------------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   test_id            693359 non-null  int64 
 1   name               693359 non-null  object
 2   item_condition_id  693359 non-null  int64 
 3   category_name      690301 non-null  object
 4   brand_name         397834 non-null  object
 5   shipping  

### Valores Nulos

In [8]:
train.isna().value_counts()

train_id  name   item_condition_id  category_name  brand_name  price  shipping  item_description
False     False  False              False          False       False  False     False               846982
                                                   True        False  False     False               629222
                                    True           True        False  False     False                 3457
                                                   False       False  False     False                 2870
                                    False          True        False  False     True                     3
                                                   False       False  False     True                     1
dtype: int64

In [9]:
test.isna().value_counts()

test_id  name   item_condition_id  category_name  brand_name  shipping  item_description
False    False  False              False          False       False     False               396431
                                                  True        False     False               293870
                                   True           True        False     False                 1655
                                                  False       False     False                 1403
dtype: int64

6327 valores nulos em ```category_name```

In [10]:
train.loc[ train.category_name.isna() == True ]['category_name'].shape

(6327,)

In [11]:
test.loc[ test.category_name.isna() == True ]['category_name'].shape

(3058,)

632682 valores nulos na coluna ```brand_name```.

Representam 42.68% das observações. 

In [12]:
train.loc[ train.brand_name.isna() == True ]['brand_name'].shape

(632682,)

In [13]:
test.loc[ test.brand_name.isna() == True ]['brand_name'].shape

(295525,)

In [14]:
print(f"Proporção de nulos no train: {round(train.loc[ train.brand_name.isna() == True ]['brand_name'].shape[0]/train.shape[0]*100, 2)}%")

Proporção de nulos no train: 42.68%


In [15]:
print(f"Proporção de nulos no test: {round(test.loc[ test.brand_name.isna() == True ]['brand_name'].shape[0]/test.shape[0]*100, 2)}%")

Proporção de nulos no test: 42.62%


In [16]:
train.loc[ train.item_description.isna() == True ]

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
511535,511535,Shoes for Michelle,4,Kids/Girls 0-24 Mos/Shoes,NaN,9.0,0,NaN
861230,861230,Lipgloss,4,Beauty/Makeup/Lips,NaN,49.0,0,NaN
1224924,1224924,Disney Minnie Head band,3,Women/Women's Accessories/Hair Accessories,Disney,9.0,0,NaN
1264242,1264242,For Bianca,3,Women/Women's Accessories/Scarves & Wraps,NaN,10.0,1,NaN


In [17]:
test.loc[ test.item_description.isna() == True ]

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description


## Separando as categorias

In [18]:
categories = train.category_name.str.split('/', expand=True)
categories.head()

,0,1,2,3,4
0,Men,Tops,T-shirts,None,None
1,Electronics,Computers & Tablets,Components & Parts,None,None
2,Women,Tops & Blouses,Blouse,None,None
3,Home,Home Décor,Home Décor Accents,None,None
4,Women,Jewelry,Necklaces,None,None


In [19]:
categories_test = test.category_name.str.split('/', expand=True)
categories_test.head()

,0,1,2,3,4
0,Women,Jewelry,Rings,None,None
1,Other,Office supplies,Shipping Supplies,None,None
2,Vintage & Collectibles,Bags and Purses,Handbag,None,None
3,Women,Sweaters,Cardigan,None,None
4,Other,Books,Religion & Spirituality,None,None


In [20]:
categories.columns = ['main_cat', 'sub_cat_1', 'sub_cat_2', 'sub_cat_3', 'sub_cat_4']
categories.head()

,main_cat,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_4
0,Men,Tops,T-shirts,None,None
1,Electronics,Computers & Tablets,Components & Parts,None,None
2,Women,Tops & Blouses,Blouse,None,None
3,Home,Home Décor,Home Décor Accents,None,None
4,Women,Jewelry,Necklaces,None,None


In [21]:
categories_test.columns = ['main_cat', 'sub_cat_1', 'sub_cat_2', 'sub_cat_3', 'sub_cat_4']
categories_test.head()

,main_cat,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_4
0,Women,Jewelry,Rings,None,None
1,Other,Office supplies,Shipping Supplies,None,None
2,Vintage & Collectibles,Bags and Purses,Handbag,None,None
3,Women,Sweaters,Cardigan,None,None
4,Other,Books,Religion & Spirituality,None,None


In [22]:
train = pd.concat([train, categories], axis=1)
test = pd.concat([test, categories_test], axis=1)

In [23]:
train.head(0)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_4


In [24]:
test.head(0)

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_4


In [25]:
train.shape

(1482535, 13)

In [26]:
test.shape

(693359, 12)

In [27]:
train.drop('category_name', axis=1, inplace=True)
train.columns

Index(['train_id', 'name', 'item_condition_id', 'brand_name', 'price',
       'shipping', 'item_description', 'main_cat', 'sub_cat_1', 'sub_cat_2',
       'sub_cat_3', 'sub_cat_4'],
      dtype='object')

In [28]:
test.drop('category_name', axis=1, inplace=True)
test.columns

Index(['test_id', 'name', 'item_condition_id', 'brand_name', 'shipping',
       'item_description', 'main_cat', 'sub_cat_1', 'sub_cat_2', 'sub_cat_3',
       'sub_cat_4'],
      dtype='object')

In [29]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   train_id           1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   brand_name         849853 non-null   object 
 4   price              1482535 non-null  float64
 5   shipping           1482535 non-null  int64  
 6   item_description   1482531 non-null  object 
 7   main_cat           1476208 non-null  object 
 8   sub_cat_1          1476208 non-null  object 
 9   sub_cat_2          1476208 non-null  object 
 10  sub_cat_3          4389 non-null     object 
 11  sub_cat_4          3059 non-null     object 
dtypes: float64(1), int64(3), object(8)
memory usage: 135.7+ MB


In [30]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   test_id            693359 non-null  int64 
 1   name               693359 non-null  object
 2   item_condition_id  693359 non-null  int64 
 3   brand_name         397834 non-null  object
 4   shipping           693359 non-null  int64 
 5   item_description   693359 non-null  object
 6   main_cat           690301 non-null  object
 7   sub_cat_1          690301 non-null  object
 8   sub_cat_2          690301 non-null  object
 9   sub_cat_3          2046 non-null    object
 10  sub_cat_4          1419 non-null    object
dtypes: int64(3), object(8)
memory usage: 58.2+ MB


In [31]:
train = train.drop(columns=['sub_cat_3',])
train = train.drop(columns=['sub_cat_4',])
test = test.drop(columns=['sub_cat_3',])
test = test.drop(columns=['sub_cat_4',])

train = train.rename(columns = {'train_id':'id'})
test = test.rename(columns = {'test_id':'id'})
drop_train = ['id','is_train']
drop_test = ['id','is_train']
value_vars = ['item_condition_id','shipping']
category_vars = ['name','main_cat','sub_cat_1','sub_cat_2','brand_name','item_description']


print(train.isna())
print(test.isna())
for v in value_vars:
    print(v)
    print(train[train[v].isna()])

for c in category_vars:
    print(c)
    print(train[train[c].isna()])

train = train.fillna('Blank') 
test = test.fillna('Blank')   

            id   name  item_condition_id  brand_name  price  shipping  \
0        False  False              False        True  False     False   
1        False  False              False       False  False     False   
2        False  False              False       False  False     False   
3        False  False              False        True  False     False   
4        False  False              False        True  False     False   
...        ...    ...                ...         ...    ...       ...   
1482530  False  False              False       False  False     False   
1482531  False  False              False       False  False     False   
1482532  False  False              False        True  False     False   
1482533  False  False              False        True  False     False   
1482534  False  False              False        True  False     False   

         item_description  main_cat  sub_cat_1  sub_cat_2  
0                   False     False      False      False  
1  

In [32]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   id                 1482535 non-null  int64  
 1   name               1482535 non-null  object 
 2   item_condition_id  1482535 non-null  int64  
 3   brand_name         1482535 non-null  object 
 4   price              1482535 non-null  float64
 5   shipping           1482535 non-null  int64  
 6   item_description   1482535 non-null  object 
 7   main_cat           1482535 non-null  object 
 8   sub_cat_1          1482535 non-null  object 
 9   sub_cat_2          1482535 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 113.1+ MB


In [33]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 693359 non-null  int64 
 1   name               693359 non-null  object
 2   item_condition_id  693359 non-null  int64 
 3   brand_name         693359 non-null  object
 4   shipping           693359 non-null  int64 
 5   item_description   693359 non-null  object
 6   main_cat           693359 non-null  object
 7   sub_cat_1          693359 non-null  object
 8   sub_cat_2          693359 non-null  object
dtypes: int64(3), object(6)
memory usage: 47.6+ MB


## Conversão de texto para nº

In [34]:
# Combine teste e treine uma vez e altere o tipo de dados de uma só vez
# adiciona uma coluna para dividir em ambos os conjuntos
train['is_train'] = 1  # aki vai colocar a coluna is_train =1 no train, para diferenciar quando juntar os 2 df para transformar em texto e depois separar 
test['is_train'] = 0   # aki vai colocar a coluna is_train =1 no test, para diferenciar quando juntar os 2 df para transformar em texto e depois separar 

# define concatenação
combine = pd.concat([train.drop(['price'], axis=1),test],axis=0)
display(combine.head())

,id,name,item_condition_id,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,is_train
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Blank,1,No description yet,Men,Tops,T-shirts,1
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,1
2,2,AVA-VIV Blouse,1,Target,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,1
3,3,Leather Horse Statues,1,Blank,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,1
4,4,24K GOLD plated rose,1,Blank,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,1


In [35]:
# altera o tipo de dados de variáveis ​​categóricas e convertendo para nº
#name
combine.name = combine.name.astype('category')
#brand_name         
combine.brand_name = combine.brand_name.astype('category')
#item_description   
combine.item_description = combine.item_description.astype('category')
#main_cat           
combine.main_cat = combine.main_cat.astype('category')
#sub_cat_1          
combine.sub_cat_1 = combine.sub_cat_1.astype('category')
#sub_cat_2          
combine.sub_cat_2  = combine.sub_cat_2 .astype('category')

display(combine.head())

,id,name,item_condition_id,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,is_train
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Blank,1,No description yet,Men,Tops,T-shirts,1
1,1,Razer BlackWidow Chroma Keyboard,3,Razer,0,This keyboard is in great condition and works ...,Electronics,Computers & Tablets,Components & Parts,1
2,2,AVA-VIV Blouse,1,Target,1,Adorable top with a hint of lace and a key hol...,Women,Tops & Blouses,Blouse,1
3,3,Leather Horse Statues,1,Blank,1,New with tags. Leather horses. Retail for [rm]...,Home,Home Décor,Home Décor Accents,1
4,4,24K GOLD plated rose,1,Blank,0,Complete with certificate of authenticity,Women,Jewelry,Necklaces,1


In [36]:
# Codifique a string de combinação com ".cat.codes"
combine.name = combine.name.cat.codes
combine.brand_name = combine.brand_name.cat.codes
combine.item_description = combine.item_description.cat.codes
combine.main_cat = combine.main_cat.cat.codes
combine.sub_cat_1 = combine.sub_cat_1.cat.codes
combine.sub_cat_2 = combine.sub_cat_2.cat.codes

In [37]:
display(combine.info())
display(combine.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175894 entries, 0 to 693358
Data columns (total 10 columns):
 #   Column             Dtype
---  ------             -----
 0   id                 int64
 1   name               int32
 2   item_condition_id  int64
 3   brand_name         int16
 4   shipping           int64
 5   item_description   int32
 6   main_cat           int8 
 7   sub_cat_1          int8 
 8   sub_cat_2          int16
 9   is_train           int64
dtypes: int16(2), int32(2), int64(4), int8(2)
memory usage: 112.1 MB


None

,id,name,item_condition_id,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,is_train
0,0,916335,3,646,1,1172054,6,103,774,1
1,1,1292428,3,3889,0,1585540,2,31,216,1
2,2,131013,1,4588,1,167133,10,104,98,1
3,3,802671,1,646,1,1136644,4,56,411,1
4,4,65051,1,646,0,531910,10,59,543,1


In [38]:

## Modelando os dados

# Separa o df concatenado , usando a coluna is_train criada anteriormente apenas para isso
df_test = combine.loc[combine['is_train'] == 0]
df_train = combine.loc[combine['is_train'] == 1]

print(df_train.info())
print(df_test.info())

display(df_train.head())
display(df_test.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1482535 entries, 0 to 1482534
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   id                 1482535 non-null  int64
 1   name               1482535 non-null  int32
 2   item_condition_id  1482535 non-null  int64
 3   brand_name         1482535 non-null  int16
 4   shipping           1482535 non-null  int64
 5   item_description   1482535 non-null  int32
 6   main_cat           1482535 non-null  int8 
 7   sub_cat_1          1482535 non-null  int8 
 8   sub_cat_2          1482535 non-null  int16
 9   is_train           1482535 non-null  int64
dtypes: int16(2), int32(2), int64(4), int8(2)
memory usage: 76.3 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 693359 entries, 0 to 693358
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   id                 693359 no

,id,name,item_condition_id,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,is_train
0,0,916335,3,646,1,1172054,6,103,774,1
1,1,1292428,3,3889,0,1585540,2,31,216,1
2,2,131013,1,4588,1,167133,10,104,98,1
3,3,802671,1,646,1,1136644,4,56,411,1
4,4,65051,1,646,0,531910,10,59,543,1


,id,name,item_condition_id,brand_name,shipping,item_description,main_cat,sub_cat_1,sub_cat_2,is_train
0,0,323913,1,646,1,1395408,10,59,667,0
1,1,65692,1,646,1,95839,7,72,701,0
2,2,410310,1,1094,1,383740,9,7,383,0
3,3,544668,2,646,0,34209,10,97,167,0
4,4,810652,3,646,1,1316301,7,15,662,0


## Montagem do treino e teste

In [39]:
#separando o X e o Y

x = df_train
y = train['price']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x)

#Como estamos lidando com valores monetários desta vez, usaremos uma função log para transformação não linear p127
y = y.apply(lambda x: np.log(x) if x>0 else x)


x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state=42)



## Montagem do regressor

In [43]:
#impport de biblioteca 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, mean_squared_log_error


#Criando e aprendendo modelos
model = RandomForestRegressor(n_jobs=-1,random_state = 42 ,min_samples_leaf=2, n_estimators=250)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

#Exibição de metricas
print(f'Score do treino: {model.score(x_train, y_train):.3f}')
print(f'Valor R² do teste: {r2_score(y_test, y_pred):.3f}')
print(f'Valor MAE do teste: {mean_absolute_error(y_test, y_pred):.3f}')
print(f'RMSE do teste: {mean_squared_error(y_test, y_pred):.3f}')
print(f'RMSLE: {mean_squared_log_error(y_test, y_pred):.3f}')

Score do treino: 0.872
Valor R² do teste: 0.489
Valor MAE do teste: 0.422
RMSE do teste: 0.322
RMSLE: 0.022


In [44]:
#impport de biblioteca 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, accuracy_score, mean_squared_log_error

#Criando e aprendendo modelos
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)


#Exibição de metricas
print(f'Score do treino: {lr.score(x_train, y_train):.3f}')
print(f'Valor R² do teste: {r2_score(y_test, y_pred):.3f}')
print(f'Valor MAE do teste: {mean_absolute_error(y_test, y_pred):.3f}')
print(f'RMSE do teste: {mean_squared_error(y_test, y_pred):.3f}')
print(f'RMSLE: {mean_squared_log_error(y_test, y_pred):.3f}')

Score do treino: 0.089
Valor R² do teste: 0.087
Valor MAE do teste: 0.589
RMSE do teste: 0.575
RMSLE: 0.038


In [45]:
#impport de biblioteca 
from sklearn.datasets import load_diabetes
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor


#Criando e aprendendo modelos
reg1 = GradientBoostingRegressor(random_state=1)
reg2 = model
reg3 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('lr', reg3)])
ereg = ereg.fit(x_train, y_train)

y_pred = ereg.predict(x_test)

#Exibição de metricas
print(f'Score do treino: {ereg.score(x_train, y_train):.3f}')
print(f'Valor R² do teste: {r2_score(y_test, y_pred):.3f}')
print(f'Valor MAE do teste: {mean_absolute_error(y_test, y_pred):.3f}')
print(f'RMSE do teste: {mean_squared_error(y_test, y_pred):.3f}')
print(f'RMSLE: {mean_squared_log_error(y_test, y_pred):.3f}')

Score do treino: 0.523
Valor R² do teste: 0.368
Valor MAE do teste: 0.486
RMSE do teste: 0.398
RMSLE: 0.027
